# script to annotate GWAS files with QTLs

In [3]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import stats, special
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns

from functools import reduce

plt.rcParams['svg.fonttype'] = 'none'

base_dir = "http://bartzabel.ls.manchester.ac.uk/orozcolab/SNP2Mechanism/"

## load all the individual QTL files

In [4]:
gtf_annotation_df = pd.read_pickle(f"{base_dir}/metadata/gencode_gtf.pickle")
gtf_transcripts = gtf_annotation_df[(gtf_annotation_df["feature"] == "transcript") & (gtf_annotation_df["transcript_type"] == "protein_coding")].dropna(axis=1, how='all')
gtf_transcripts["gene_id"] = gtf_transcripts["gene_id"].str.split(".").str[0]
gtf_transcripts["transcript_id"] = gtf_transcripts["transcript_id"].str.split(".").str[0]
gtf_transcripts["TSS_start"] = gtf_transcripts.apply(lambda x: int(x["start"]) if x["strand"] == "+" else int(x["end"]) ,axis = 1)
gene_mapper = dict(zip(gtf_transcripts['gene_id'], gtf_transcripts['gene_name']))

In [5]:
RNA_nominal_CD4 = pd.read_csv(f"{base_dir}/QTLs/RNA/RNA_nominal_CD4_merged.txt", sep = " ")
RNA_nominal_CD8 = pd.read_csv(f"{base_dir}/QTLs/RNA/RNA_nominal_CD8_merged.txt", sep = " ")
RNA_permuted_CD4 = pd.read_csv(f"{base_dir}/QTLs/RNA/RNA_permuted_CD4_FDR.txt", sep = " ")
RNA_permuted_CD8 = pd.read_csv(f"{base_dir}/QTLs/RNA/RNA_permuted_CD8_FDR.txt", sep = " ")# RNA_permuted_CD8 = RNA_permuted_CD8[RNA_permuted_CD8["adj_beta_pval"] < 0.1]
# RNA_permuted_CD4 = RNA_permuted_CD4[RNA_permuted_CD4["adj_beta_pval"] < 0.1]
RNA_nominal_CD8 = RNA_nominal_CD8[RNA_nominal_CD8["nom_pval"] < 0.001]
RNA_nominal_CD4 = RNA_nominal_CD4[RNA_nominal_CD4["nom_pval"] < 0.001]

In [6]:
RNA_permuted_CD4 = RNA_permuted_CD4[["phe_id", "var_id", "nom_pval", "slope", "adj_beta_pval", "FDR"]]
RNA_permuted_CD4 = RNA_permuted_CD4.rename(columns={c: c+'_RNA_perm_CD4' for c in RNA_permuted_CD4.columns if c not in ['phe_id', 'var_id']})
RNA_permuted_CD8 = RNA_permuted_CD8[["phe_id", "var_id", "nom_pval", "slope", "adj_beta_pval", "FDR"]]
RNA_permuted_CD8 = RNA_permuted_CD8.rename(columns={c: c+'_RNA_perm_CD8' for c in RNA_permuted_CD8.columns if c not in ['phe_id', 'var_id']})
RNA_nominal_CD4 = RNA_nominal_CD4[["phe_id", "var_id", "nom_pval", "slope"]]
RNA_nominal_CD4 = RNA_nominal_CD4.rename(columns={c: c+'_RNA_nom_CD4' for c in RNA_nominal_CD4.columns if c not in ['phe_id', 'var_id']})
RNA_nominal_CD8 = RNA_nominal_CD8[["phe_id", "var_id", "nom_pval", "slope"]]
RNA_nominal_CD8 = RNA_nominal_CD8.rename(columns={c: c+'_RNA_nom_CD8' for c in RNA_nominal_CD8.columns if c not in ['phe_id', 'var_id']})

In [7]:
ATAC_nominal_CD4 = pd.read_csv(f"{base_dir}/QTLs/ATAC/ATAC_nominal_CD4_merged.txt", sep = " ")
ATAC_nominal_CD8 = pd.read_csv(f"{base_dir}/QTLs/ATAC/ATAC_nominal_CD8_merged.txt", sep = " ")
ATAC_permuted_CD4 = pd.read_csv(f"{base_dir}/QTLs/ATAC/ATAC_permuted_CD4_FDR.txt", sep = " ")
ATAC_permuted_CD8 = pd.read_csv(f"{base_dir}/QTLs/ATAC/ATAC_permuted_CD8_FDR.txt", sep = " ")# ATAC_permuted_CD8 = ATAC_permuted_CD8[ATAC_permuted_CD8["adj_beta_pval"] < 0.1]
# ATAC_permuted_CD4 = ATAC_permuted_CD4[ATAC_permuted_CD4["adj_beta_pval"] < 0.1]
ATAC_nominal_CD8 = ATAC_nominal_CD8[ATAC_nominal_CD8["nom_pval"] < 0.001]
ATAC_nominal_CD4 = ATAC_nominal_CD4[ATAC_nominal_CD4["nom_pval"] < 0.001]

In [8]:
ATAC_permuted_CD4 = ATAC_permuted_CD4[["phe_chr","phe_from","phe_to","phe_id", "dist_phe_var", "var_id", "nom_pval", "slope", "adj_beta_pval", "FDR"]]
ATAC_permuted_CD4 = ATAC_permuted_CD4.rename(columns={c: c+'_ATAC_perm_CD4' for c in ATAC_permuted_CD4.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})
ATAC_permuted_CD8 = ATAC_permuted_CD8[["phe_chr","phe_from","phe_to","phe_id", "dist_phe_var", "var_id", "nom_pval", "slope", "adj_beta_pval", "FDR"]]
ATAC_permuted_CD8 = ATAC_permuted_CD8.rename(columns={c: c+'_ATAC_perm_CD8' for c in ATAC_permuted_CD8.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})
ATAC_nominal_CD4 = ATAC_nominal_CD4[["phe_chr","phe_from","phe_to","phe_id", "dist_phe_var", "var_id", "nom_pval", "slope"]]
ATAC_nominal_CD4 = ATAC_nominal_CD4.rename(columns={c: c+'_ATAC_nom_CD4' for c in ATAC_nominal_CD4.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})
ATAC_nominal_CD8 = ATAC_nominal_CD8[["phe_chr","phe_from","phe_to","phe_id", "dist_phe_var", "var_id", "nom_pval", "slope"]]
ATAC_nominal_CD8 = ATAC_nominal_CD8.rename(columns={c: c+'_ATAC_nom_CD8' for c in ATAC_nominal_CD8.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})

In [9]:
ins_nominal_CD4 = pd.read_csv(f"{base_dir}/QTLs/HiC/ins_nominal_CD4_merged.txt", sep = " ")
ins_nominal_CD8 = pd.read_csv(f"{base_dir}/QTLs/HiC/ins_nominal_CD8_merged.txt", sep = " ")
ins_permuted_CD4 = pd.read_csv(f"{base_dir}/QTLs/HiC/ins_permuted_CD4_FDR.txt", sep = " ")
ins_permuted_CD8 = pd.read_csv(f"{base_dir}/QTLs/HiC/ins_permuted_CD8_FDR.txt", sep = " ")
# ins_permuted_CD8 = ins_permuted_CD8[ins_permuted_CD8["adj_beta_pval"] < 0.1]
# ins_permuted_CD4 = ins_permuted_CD4[ins_permuted_CD4["adj_beta_pval"] < 0.1]
ins_nominal_CD8 = ins_nominal_CD8[ins_nominal_CD8["nom_pval"] < 0.001]
ins_nominal_CD4 = ins_nominal_CD4[ins_nominal_CD4["nom_pval"] < 0.001]

In [10]:
ins_permuted_CD4 = ins_permuted_CD4[["phe_chr","phe_from","phe_to","phe_id", "var_id", "nom_pval", "slope", "adj_beta_pval", "FDR"]]
ins_permuted_CD4 = ins_permuted_CD4.rename(columns={c: c+'_ins_perm_CD4' for c in ins_permuted_CD4.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})
ins_permuted_CD8 = ins_permuted_CD8[["phe_chr","phe_from","phe_to","phe_id", "var_id", "nom_pval", "slope", "adj_beta_pval", "FDR"]]
ins_permuted_CD8 = ins_permuted_CD8.rename(columns={c: c+'_ins_perm_CD8' for c in ins_permuted_CD8.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})
ins_nominal_CD4 = ins_nominal_CD4[["phe_chr","phe_from","phe_to","phe_id", "var_id", "nom_pval", "slope"]]
ins_nominal_CD4 = ins_nominal_CD4.rename(columns={c: c+'_ins_nom_CD4' for c in ins_nominal_CD4.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})
ins_nominal_CD8 = ins_nominal_CD8[["phe_chr","phe_from","phe_to","phe_id", "var_id", "nom_pval", "slope"]]
ins_nominal_CD8 = ins_nominal_CD8.rename(columns={c: c+'_ins_nom_CD8' for c in ins_nominal_CD8.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})

In [11]:
loop_nominal_CD4 = pd.read_csv(f"{base_dir}/QTLs/HiC/loop_nominal_CD4_merged.txt", sep = " ")
loop_nominal_CD8 = pd.read_csv(f"{base_dir}/QTLs/HiC/loop_nominal_CD8_merged.txt", sep = " ")
loop_permuted_CD4 = pd.read_csv(f"{base_dir}/QTLs/HiC/loop_permuted_CD4_FDR.txt", sep = " ")
loop_permuted_CD8 = pd.read_csv(f"{base_dir}/QTLs/HiC/loop_permuted_CD8_FDR.txt", sep = " ")
# loop_permuted_CD8 = loop_permuted_CD8[loop_permuted_CD8["adj_beta_pval"] < 0.1]
# loop_permuted_CD4 = loop_permuted_CD4[loop_permuted_CD4["adj_beta_pval"] < 0.1]
loop_nominal_CD8 = loop_nominal_CD8[loop_nominal_CD8["nom_pval"] < 0.001]
loop_nominal_CD4 = loop_nominal_CD4[loop_nominal_CD4["nom_pval"] < 0.001]

In [12]:
loop_permuted_CD4 = loop_permuted_CD4[["phe_chr","phe_from","phe_to","phe_id", "var_id", "nom_pval", "slope", "adj_beta_pval", "FDR"]]
loop_permuted_CD4 = loop_permuted_CD4.rename(columns={c: c+'_loop_perm_CD4' for c in loop_permuted_CD4.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})
loop_permuted_CD8 = loop_permuted_CD8[["phe_chr","phe_from","phe_to","phe_id", "var_id", "nom_pval", "slope", "adj_beta_pval", "FDR"]]
loop_permuted_CD8 = loop_permuted_CD8.rename(columns={c: c+'_loop_perm_CD8' for c in loop_permuted_CD8.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})
loop_nominal_CD4 = loop_nominal_CD4[["phe_chr","phe_from","phe_to","phe_id", "var_id", "nom_pval", "slope"]]
loop_nominal_CD4 = loop_nominal_CD4.rename(columns={c: c+'_loop_nom_CD4' for c in loop_nominal_CD4.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})
loop_nominal_CD8 = loop_nominal_CD8[["phe_chr","phe_from","phe_to","phe_id", "var_id", "nom_pval", "slope"]]
loop_nominal_CD8 = loop_nominal_CD8.rename(columns={c: c+'_loop_nom_CD8' for c in loop_nominal_CD8.columns if c not in ['phe_id', 'var_id',"phe_chr","phe_from","phe_to"]})

## prepare tables so that we can actually annotate the GWAS files

In [13]:
dataframe_list = [RNA_permuted_CD8, RNA_nominal_CD8, RNA_permuted_CD4, RNA_nominal_CD4]
RNA_merged_df = reduce(lambda left, right: left.merge(right, on=["phe_id", "var_id"], how = "outer"), dataframe_list)

In [14]:
RNA_merged_df["gene_name"] = RNA_merged_df['phe_id'].map(gene_mapper)

In [15]:
dataframe_list = [ATAC_permuted_CD8, ATAC_nominal_CD8, ATAC_permuted_CD4, ATAC_nominal_CD4]
ATAC_merged_df = reduce(lambda left, right: left.merge(right, on=["phe_id", "var_id","phe_chr","phe_from","phe_to"], how = "outer"), dataframe_list)

In [16]:
dataframe_list = [ins_permuted_CD8, ins_nominal_CD8, ins_permuted_CD4, ins_nominal_CD4]
ins_merged_df = reduce(lambda left, right: left.merge(right, on=["phe_id", "var_id","phe_chr","phe_from","phe_to"], how = "outer"), dataframe_list)

In [17]:
dataframe_list = [loop_permuted_CD8, loop_nominal_CD8, loop_permuted_CD4, loop_nominal_CD4]
loop_merged_df = reduce(lambda left, right: left.merge(right, on=["phe_id", "var_id","phe_chr","phe_from","phe_to"], how = "outer"), dataframe_list)

In [18]:
RNA_merged_df = RNA_merged_df.rename(columns = {"phe_id" : "gene_id"})
ATAC_merged_df = ATAC_merged_df.rename(columns = {"phe_id" : "peak_id", "phe_chr":"peak_chr","phe_from":"peak_start","phe_to":"peak_end"})
ins_merged_df = ins_merged_df.rename(columns = {"phe_id" : "bin_id", "phe_chr":"bin_chr","phe_from":"bin_start","phe_to":"bin_end"})
loop_merged_df = loop_merged_df.rename(columns = {"phe_id" : "loop_id", "phe_chr":"loop_chr","phe_from":"loop_A_start","phe_to":"loop_B_end"})

## annotate
this creates a few files. the first one contains all the QTL together, whilst the others are one file for each QTL. 

The way it works it that it creates a bit of a long format file, in which for each variant we will have one row for each hit. Also because multiple variants will be linked together when it does hit all the variants will have duplicate rows, which can make visualizing in excel a bit difficult.

In [19]:
def annotate(file, name):
    snps_df  = pd.read_csv(file, sep = "\t", header = None)
    snps_df.columns = "chr start end name score".split()
    snps_df["loci"] = snps_df["name"].str.split("_").str[-1]
    snps_df["snp"] = snps_df["name"].str.split("_").str[0]

    annotated_snps_df = snps_df.merge(RNA_merged_df.rename(columns={"var_id":"snp"}), 
                                          on = "snp", how = "left")
    annotated_snps_df = annotated_snps_df.merge(ATAC_merged_df.rename(columns={"var_id":"snp"}),
                                          on = "snp", how = "left")
    annotated_snps_df = annotated_snps_df.merge(ins_merged_df.rename(columns={"var_id":"snp"}),
                                          on = "snp", how = "left")
    annotated_snps_df = annotated_snps_df.merge(loop_merged_df.rename(columns={"var_id":"snp"}),
                                          on = "snp", how = "left")
    annotated_snps_df = annotated_snps_df.drop(columns = ["nom_pval_RNA_perm_CD8", "slope_RNA_perm_CD8","nom_pval_RNA_perm_CD4", "slope_RNA_perm_CD4",
                                                          "nom_pval_ATAC_perm_CD8", "slope_ATAC_perm_CD8","nom_pval_ATAC_perm_CD4", "slope_ATAC_perm_CD4",
                                                          "nom_pval_ins_perm_CD8", "slope_ins_perm_CD8","nom_pval_ins_perm_CD4", "slope_ins_perm_CD4",
                                                          "nom_pval_loop_perm_CD8", "slope_loop_perm_CD8","nom_pval_loop_perm_CD4", "slope_loop_perm_CD4",
                                                          ])
    annotated_snps_df.to_csv(f"output_tables/{name}_QTL_mapped.csv")


    snps_df.merge(RNA_merged_df.rename(columns={"var_id":"snp"}), 
                                            on = "snp", how = "left").to_csv(
        f"output_tables/{name}_RNA_QTL.csv")
    snps_df.merge(ATAC_merged_df.rename(columns={"var_id":"snp"}), 
                                            on = "snp", how = "left").to_csv(
        f"output_tables/{name}_ATAC_QTL.csv")
    snps_df.merge(ins_merged_df.rename(columns={"var_id":"snp"}), 
                                            on = "snp", how = "left").to_csv(
        f"output_tables/{name}_ins_QTL.csv")
    snps_df.merge(loop_merged_df.rename(columns={"var_id":"snp"}), 
                                            on = "snp", how = "left").to_csv(
        f"output_tables/{name}_loop_QTL.csv")



In [20]:
annotate(f"{base_dir}/other_tracks/GWAS/tsoi2017_LD_0.8_hg38.bed", "psoriasis_tsoi2017")
annotate(f"{base_dir}/other_tracks/GWAS/RAmetagwas_all_hg38.ld.bed", "RAmeta")
annotate(f"{base_dir}/other_tracks/GWAS/PsA_vs_controls_metagwas_significant.ld.hg38.bed", "PsA_meta")
annotate(f"{base_dir}/other_tracks/GWAS/suggestive_snps_hg38_ld.bed", "JIA_suggestive")
annotate(f"{base_dir}/other_tracks/GWAS/JIA_credible_snps_hg38.bed", "JIA_credible")
annotate(f"{base_dir}/other_tracks/GWAS/elena_hg38.ld.bed", "SSc_elena")